In [1]:
import pickle
import tarfile
import chess.pgn
import io
from lcztools import LeelaBoard, load_network
from torch import optim, nn
import numpy as np
import torch

net = load_network()

for name, param in net.model.named_parameters():
    # print(name)
    if ('_val' in name) and not ('conv1_bn.weight' in name):
        print('+', name)
        param.requires_grad = True
    else:
        # print('-', name)
        param.requires_grad = False

optimizer = optim.Adam(net.model.parameters(), lr = 0.001)
criterion = nn.MSELoss()

losses = []

def do_step(training_game):
    if training_game[1] == '1/2-1/2':
        result = 0
    elif training_game[1] == '1-0':
        result = 1
    elif training_game[1] == '0-1':
        result = -1
    elif training_game[1] == '*':
        return
    features_stack = []
    results = []
    for compressed_features in training_game[0]:
        features_stack.append(LeelaBoard.decompress_features(compressed_features))
        results.append(result)
        result *= -1
    features_stack = np.stack(features_stack)
    
    optimizer.zero_grad()
    pols, vals = net.model(features_stack)
    results = torch.Tensor(results).view(-1, 1).cuda()
    loss = criterion(vals, results)
    losses.append(loss.item())
    if len(losses)==100:
        print(sum(losses)/len(losses))
        losses.clear()
    loss.backward()
    optimizer.step()    

with tarfile.open('training_data.tgz') as f:
    for idx, member in enumerate(f):
        if member.isfile():
            training_game = pickle.load(f.extractfile(member))
            do_step(training_game)
# training_data = []

Loading network using backend=pytorch_train_cuda, policy_softmax_temp=2.2
Channels 256
Blocks 20
Enabling CUDA!
+ conv_val.conv1.weight
+ conv_val.conv1_bn.bias
+ affine_val_1.weight
+ affine_val_1.bias
+ affine_val_2.weight
+ affine_val_2.bias
0.3443026130460203
0.32471403952687977
0.35659265039488675
0.3510569354146719
0.36359788935631515
0.33464580290019513
0.3016416857950389
0.31575789995491504
0.3402637189626694
0.3824492286145687
0.3226519647426903
0.35024036847054957
0.33501042999327185
0.33703164126724006
0.32323848369531333
0.3279957647807896
0.34316440649330615
0.31050884516909716
0.3425578296929598
0.3386318416148424
0.3301412049122155
0.36275455344468355
0.34623996417969466
0.3436740582622588
0.37055384036153555
0.30501582579687236
0.3026069594547153
0.2946700772084296
0.35004374595358967
0.36694354539737106
0.3233475613873452
0.3395469257887453
0.33980841885786506
0.27501192246796563
0.3339210773049854
0.31620211266214027
0.32661044409498574
0.32781210164073854
0.346956444

In [ ]:


# def gen_eval_result(training_game):
if 1:
    if training_game[1] == '1/2-1/2':
        result = 0
    elif training_game[1] == '1-0':
        result = 1
    elif training_game[1] == '0-1':
        result = -1
    features_stack = []
    results = []
    for compressed_features in training_game[0]:
        features_stack.append(LeelaBoard.decompress_features(compressed_features))
        results.append(result)
        result *= -1
    features_stack = np.stack(features_stack)
    
    optimizer.zero_grad()
    evs = net.model(features_stack)[1]
    results = torch.Tensor(results).view(-1, 1).cuda()
    loss = criterion(evs, results)
    print(loss.item())
    loss.backward()
    optimizer.step()
    
# gen_eval_result(training_game)    
#         results.append(result)
#         result *= -1
    
    
        
        
#     ev = net.model(lcz_board.lcz_features())[1].item()
#     yield lcz_board, ev, result, (ev-result)**2
#     for i, move in enumerate(moves, 1):
#         result = result*-1
#         lcz_board.push(move)
#         features = lcz_board.lcz_features()
#         # features_2 = lcz_board.decompress_features(training_game[0][i])
#         # print(i, end=' ')
#         # if i==len(moves)-1:
#         #     break
#         # assert((features==features_2).all())
#         ev = net.model(lcz_board.lcz_features())[1].item()
#         yield lcz_board, ev, result, (ev-result)**2
# from IPython.display import display

# for lcz_board, ev, result, mse in gen_eval_result(training_game):
#     display(lcz_board)
#     print(ev, result, mse)
        

In [ ]:
for name, param in net.model.named_parameters():
    # print(name)
    if '_val' in name:
        print('+', name)
        param.requires_grad = True
    else:
        print('-', name)
        param.requires_grad = False


In [ ]:
import numpy as np

def gen_eval_result(training_game):
    pgn_string = training_game[2]
    if training_game[1] == '1/2-1/2':
        result = 0
    elif training_game[1] == '1-0':
        result = 1
    elif training_game[1] == '0-1':
        result = -1    
    pgn_game = chess.pgn.read_game(io.StringIO(pgn_string))
    moves = [move for move in pgn_game.main_line()]
    lcz_board = LeelaBoard()
    ev = net.model(lcz_board.lcz_features())[1].item()
    yield lcz_board, ev, result, (ev-result)**2
    for i, move in enumerate(moves, 1):
        result = result*-1
        lcz_board.push(move)
        features = lcz_board.lcz_features()
        # features_2 = lcz_board.decompress_features(training_game[0][i])
        # print(i, end=' ')
        # if i==len(moves)-1:
        #     break
        # assert((features==features_2).all())
        ev = net.model(lcz_board.lcz_features())[1].item()
        yield lcz_board, ev, result, (ev-result)**2
from IPython.display import display

for lcz_board, ev, result, mse in gen_eval_result(training_game):
    display(lcz_board)
    print(ev, result, mse)
        

In [ ]:
np.array_equal(features_2, features)
# (features_2==features)

In [ ]:
for i in range(112):
    for x in range(8):
        for y in range(8):
            if features[i,x,y] != features_2[i, x, y]:
                print(i,x,y,features[i,x,y], features_2[i,x,y])

In [ ]:
f1 = lcz_board.decompress_features(lcz_board.compress_features(features))

In [ ]:
np.array_equal(f1, features)

In [ ]:
param

In [ ]:
from lcztools import LeelaBoard, load_network
import torch

net = load_network()

# def save_weights_file(self, filename):
#     LEELA_WEIGHTS_VERSION = '2'
#     lines = [LEELA_WEIGHTS_VERSION]
#     parameters = []
#     for module_name, module in self.named_modules():
#         print('.', end='')
#         class_name = module.__class__.__name__
#         for typ in ('weight', 'bias', 'running_mean', 'running_var'):
#             param = getattr(module, typ, None)
#             if param is not None:
#                 # print(module_name, class_name, typ)
#                 if class_name == 'CenteredBatchNorm2d' and typ=='bias':
#                     # print('-- updating')
#                     std = torch.sqrt(getattr(module, 'running_var').cpu().detach() + 1e-5)
#                     param_data = param.cpu().detach()*std
#                 else:
#                     param_data = param.cpu().detach()
#                 lines.append(' '.join(map(str, param_data.flatten().tolist())))
#     lines.append('')
#     with open(filename, 'w') as f:
#         for line in lines:
#             f.write(line)
#             f.write('\n')



In [ ]:
net.model.cuda()
p = list(net.model.parameters())[0]
p.cpu().detach()

In [ ]:
p.detach?

In [2]:
net.model.save_weights_file('/home/trevor/projects/lczero/weights/txt/test_weights.txt')

Saving weights file:
.................................................................................................................Done saving weights!


In [ ]:
sum((data_real - data_test)**2)

In [ ]:
list(net.model.conv_val.conv1_bn.named_parameters())

In [ ]:
#del net.model.conv_val.conv1_bn.weight
net.model.conv_val.conv1_bn.bias

In [ ]:
net.model.__delattr__??

In [ ]:
net.model.conv_val.conv1_bn.register_parameter??